# Collaborative Filtering

## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

## Dataset Representation

In [4]:
book_df = pd.read_csv('Dataset3.csv', index_col=0)
book_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.0,5.0,4.0,NaN,1.0,3.0,4.0,1.0,5.0,NaN,...,3.0,5.0,NaN,2.0,NaN,1.0,4.0,5.0,3.0,4.0
1,NaN,NaN,5.0,3.0,4.0,1.0,4.0,NaN,NaN,1.0,...,5.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0
2,2.0,4.0,1.0,NaN,NaN,4.0,3.0,NaN,4.0,4.0,...,3.0,2.0,4.0,NaN,3.0,2.0,4.0,5.0,1.0,4.0
3,3.0,5.0,1.0,5.0,1.0,3.0,5.0,3.0,2.0,4.0,...,1.0,2.0,2.0,2.0,5.0,4.0,NaN,NaN,3.0,NaN
4,NaN,NaN,5.0,3.0,2.0,1.0,4.0,NaN,4.0,1.0,...,3.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,5.0,5.0,2.0,NaN,NaN,3.0,2.0,4.0,5.0,...,4.0,1.0,NaN,5.0,NaN,2.0,5.0,4.0,3.0,4.0
96,3.0,4.0,NaN,3.0,NaN,3.0,NaN,1.0,3.0,2.0,...,4.0,3.0,1.0,4.0,5.0,5.0,3.0,2.0,4.0,NaN
97,2.0,2.0,2.0,NaN,4.0,1.0,1.0,4.0,5.0,3.0,...,4.0,4.0,5.0,2.0,5.0,4.0,2.0,NaN,4.0,4.0
98,1.0,5.0,NaN,NaN,1.0,5.0,5.0,4.0,4.0,NaN,...,3.0,2.0,5.0,4.0,NaN,3.0,3.0,5.0,3.0,2.0


## Exploratory Data Analysis

## Data Mining

In [6]:
from collaborative_filtering import CollaborativeFiltering

SyntaxError: invalid syntax (collaborative_filtering.py, line 1)

## Insights and Conclusions

## References